# ARCH and GARCH Modeling

## Introduction

ARCH modeling, often referred to as ARCH (Autoregressive Conditional Heteroskedasticity) modeling, is a statistical technique used to model time series data with time-varying volatility. It is particularly useful in financial econometrics for modeling and forecasting the volatility of asset returns.

## Key Concepts

1. **Heteroskedasticity**: This refers to the condition where the variance of the error terms in a regression model is not constant over time.
2. **Autoregressive**: This indicates that the model uses past values of the variable to predict future values.

## ARCH Model

The ARCH model was introduced by Robert Engle in 1982. The basic idea is to model the variance of the current error term as a function of the squared error terms from previous periods.

### Basic ARCH Model

The basic ARCH(q) model can be described as follows:

1. **Mean Equation**:
   
   $y_t = \mu + \epsilon_t$
   
   where $y_t$ is the time series value at time $t$ , $\mu$ is the mean, and $\epsilon_t$ is the error term (return residuals, with respect to a mean process)

2. **Variance Equation**:
   
   $\epsilon_t = \sigma_t z_t$

   $\sigma_t^2 = \alpha_0 + \alpha_1 \epsilon_{t-1}^2 + \alpha_2 \epsilon_{t-2}^2 + \ldots + \alpha_q \epsilon_{t-q}^2 = \alpha_0 +  \sum_{i=1}^q  \alpha_i \epsilon_{t-i}^2$
   
   where  $\sigma_t^2$ is the conditional variance at time $t$, $\alpha_0,\alpha_1 \ldots \alpha_q$ are parameters to be estimated, and $z_t$ is a white noise error term.

## GARCH Model

The Generalized ARCH (GARCH) model extends the ARCH model by including lagged values of the conditional variance. The GARCH(p, q) model can be described as:

1. **Mean Equation**:

   $y_t = \mu + \epsilon_t$

2. **Variance Equation**:

   $\epsilon_t = \sigma_t z_t$
   
   $\sigma_t^2 = \alpha_0 + \sum_{i=1}^{q} \alpha_i \epsilon_{t-i}^2 + \sum_{j=1}^{p} \beta_j \sigma_{t-j}^2$

   where $\beta_j$ are additional parameters to be estimated.

## Applications

- **Financial Markets**: Modeling and forecasting the volatility of stock returns, exchange rates, and other financial assets.
- **Risk Management**: Estimating Value at Risk (VaR) and other risk metrics.
- **Econometrics**: Analyzing economic time series data with changing volatility.

## Implementation

ARCH and GARCH models can be implemented using various statistical software packages, such as R, Python (with libraries like `arch` and `statsmodels`), and others.

### Example in Python

```python
import numpy as np
import pandas as pd
from arch import arch_model

# Generate some synthetic data
np.random.seed(42)
returns = np.random.normal(0, 1, 1000)

# Fit an ARCH model
model = arch_model(returns, vol='ARCH', p=1)
model_fit = model.fit()

# Print the model summary
print(model_fit.summary())

In [7]:
import numpy as np
from scipy.optimize import minimize

class ARCHModel:
    def __init__(self, returns):
        self.returns = np.array(returns)
        self.conditionnal_variance = None
        self.mu = None
        self.alpha0 = None
        self.alpha1 = None
        self.max_log_likelihood = None
       
    
    def _compute_residuals(self, mu):
        residuals = self.returns - mu
        shift_residuals = np.roll(residuals, 1)
        shift_residuals[0] = 0
        return residuals, shift_residuals 
    
    def _compute_conditional_variance(self, alpha0, alpha1, shift_residuals):
        variances = alpha0 + alpha1 * shift_residuals ** 2
        return variances
    
    def _log_normal_density(self, residual , variance):
        return -0.5 * np.log(2 * np.pi * variance) - (residual ** 2) / (2 * variance)

    def _mle_arch(self, params):
        mu, alpha0, alpha1 = params
        residuals, shift_residuals = self._compute_residuals(mu)
        conditionnal_variance = self._compute_conditional_variance(alpha0, alpha1, shift_residuals)
        log_likelihood = self._log_normal_density(residuals, conditionnal_variance)
        return -np.sum(log_likelihood)
    
    def fit(self):
        mu =  np.mean( self.returns);
        alpha0 =  np.var(self.returns);
        alpha1 = 0  
        
        initial_params = [mu, alpha0, alpha1]
        self.max_log_likelihood = -self._mle_arch(initial_params)

        print( {
            "init mu": mu,
            "init alpha0": alpha0,
            "init alpha1": alpha1,
            "init max_log_likelihood": self.max_log_likelihood
        })

        
        bounds = [(None, None), (None, None), (None, None)] 
        
        # Minimize the negative log-likelihood
        result = minimize(self._mle_arch, initial_params, bounds=bounds, method='Nelder-Mead') 
        if result.success:
            self.mu, self.alpha0, self.alpha1 = result.x
            self.max_log_likelihood = -result.fun  # Store the maximum log-likelihood
        else:
            raise ValueError("Optimization failed: " + result.message)

        self.mu, self.alpha0, self.alpha1 = result.x
        self.max_log_likelihood = -result.fun  # Store the maximum log-likelihood
        print("max_log_likelihood", self.max_log_likelihood)

    def summary(self):
        return {
            "mu": self.mu,
            "alpha0": self.alpha0,
            "alpha1": self.alpha1,
            "max_log_likelihood": self.max_log_likelihood
        }





# Generate synthetic data
np.random.seed(42)


# Initialize arrays
returns = [0.003146575,-0.017723909,0.007254689,0.014989689,
0.011250607,0.003328635,-0.005409767,0.007332392,0.001700427,
0.006432783,-0.001627508,0.001867275,-0.002928821,0.004717681,
-0.003839112,-0.010494486,-0.033525076,-0.030286122,-0.003772137,
-0.04415351,-0.008258359]

# Create the ARCH model and fit it
model = ARCHModel(returns)
model.fit()

# Print the model summary
print(model.summary())

{'init mu': -0.004761812095238095, 'init alpha0': 0.00022024427545202675, 'init alpha1': 0, 'init max_log_likelihood': 58.620410291243196}
max_log_likelihood 75.74047540068054
{'mu': -0.00825835900119399, 'alpha0': 0.00031975580147541676, 'alpha1': -0.2481686188042993, 'max_log_likelihood': 75.74047540068054}


C:\Users\BenjaminMoiroud\AppData\Local\Temp\ipykernel_19792\3864532026.py:25: RuntimeWarning: invalid value encountered in log
  return -0.5 * np.log(2 * np.pi * variance) - (residual ** 2) / (2 * variance)
C:\Users\BenjaminMoiroud\AppData\Local\Temp\ipykernel_19792\3864532026.py:25: RuntimeWarning: divide by zero encountered in log
  return -0.5 * np.log(2 * np.pi * variance) - (residual ** 2) / (2 * variance)
C:\Users\BenjaminMoiroud\AppData\Local\Temp\ipykernel_19792\3864532026.py:25: RuntimeWarning: divide by zero encountered in divide
  return -0.5 * np.log(2 * np.pi * variance) - (residual ** 2) / (2 * variance)
C:\Users\BenjaminMoiroud\AppData\Local\Temp\ipykernel_19792\3864532026.py:25: RuntimeWarning: invalid value encountered in subtract
  return -0.5 * np.log(2 * np.pi * variance) - (residual ** 2) / (2 * variance)
